In [1]:
import pandas as pd 
import yfinance as yf
from datetime import datetime, timedelta


In [2]:
SP500_Url = "https://en.wikipedia.org/wiki/Nasdaq-100"
data_table = pd.read_html(
    SP500_Url,
    storage_options={"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
                                   "(KHTML, like Gecko) Chrome/123.0 Safari/537.36"}
)
data_table

[                                     0                                      1
 0   Nasdaq-100 Index from 1985 to 2015     Nasdaq-100 Index from 1985 to 2015
 1                           Foundation         January 31, 1985; 40 years ago
 2                             Operator                           Nasdaq, Inc.
 3                            Exchanges                                 Nasdaq
 4                       Trading symbol                            ^NDX or NDX
 5                         Constituents                                    101
 6                                 Type                              Large-cap
 7                           Market cap  US$25.34 trillion (as of May 1, 2025)
 8                     Weighting method     Free-float capitalization-weighted
 9                      Related indices                   Nasdaq Financial-100
 10                             Website                  nasdaq.com/nasdaq-100,
    Category  All-Time Highs[8]       All-Time High

In [3]:
tickers = data_table[4]["Ticker"].tolist()

In [4]:
supp = ['ASML', 'AZN', 'F', 'G', 'GOOGL', 'H', 'I', 'J', 'L', 'O', 'Q', 'R', 'U', 'VRSK', 'W', 'Y']

tickers = [x for x in tickers if x not in supp]
print(len(tickers), len(supp))

97 16


In [5]:
# Extraction des données
Date = data_table[5]['Date']
Removed = data_table[5]['Removed']['Ticker']  # Supposant que 'Removed' est un DataFrame ou Series
Added = data_table[5]['Added']['Ticker']      # Supposant que 'Added' est un DataFrame ou Series

# Combinaison des données dans un DataFrame
Data_Change = pd.concat([Date, Added, Removed], axis = 1)

#Date au format Date
Data_Change['Date'] = pd.to_datetime(Data_Change['Date'])

# Changer le format de la date en 'jour/mois/année'
Data_Change['Date'] = Data_Change['Date'].dt.strftime('%Y-%m-%d')


# Renommer toutes les colonnes
Data_Change.columns = ['Date', 'Added', 'Removed']

# Mettre 'colonne_a' en tant qu'index
Data_Change.set_index('Date', inplace=True)

# Nan en vide
Data_Change = Data_Change.fillna('')

Data_Change

,Added,Removed
Date,,
2025-07-28,TRI,
2025-07-17,,ANSS
2025-05-19,SHOP,MDB
2024-12-23,PLTR,ILMN
2024-12-23,MSTR,MRNA
...,...,...
2007-07-12,FWLT,BMET
2007-06-01,CEPH,MEDI
2007-03-08,UAUA,AEOS


In [6]:
len(tickers)

97

In [7]:
# Supposons que la liste tickers ait déjà 503 éléments au départ
# tickers = ['AAPL', 'MSFT', ...]  # Exemples de tickers initiaux

# Initialiser un dictionnaire vide
dict = {}

# Boucle pour parcourir les données et mettre à jour le dictionnaire
for i in range(0, len(Data_Change)):
    
    # Ajouter les nouveaux tickers (si ce n'est pas déjà un ensemble vide)
    tickers.extend(Data_Change.Added[i])
    
    # Essayer de supprimer les tickers retirés, si possible
    try:
        tickers = [t for t in tickers if t not in Data_Change.Removed[i]]  # Supprimer les tickers retirés
    except ValueError:
        # Si l'élément à supprimer n'existe pas dans la liste, continuer sans erreur
        pass
    
    # Trier les tickers
    tickers = sorted(set(tickers))  # Utilisation de set pour éviter les doublons

    # Ajouter une **copie** de la liste `tickers` dans le dictionnaire
    dict[Data_Change.index[i]] = tickers.copy()

df = pd.DataFrame(list(dict.items()), columns=['date', 'tickers'])

df['date'] = pd.to_datetime(df['date'])

# Mettre la colonne 'date' comme index
df.set_index('date', inplace=True)

df

/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_3527/1527443372.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tickers.extend(Data_Change.Added[i])
/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_3527/1527443372.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tickers = [t for t in tickers if t not in Data_Change.Removed[i]]  # Supprimer les tickers retirés


,tickers
date,
2025-07-28,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."


In [8]:
# Récupère l'index en liste
new_index = list(df.index)

# Remplace la première valeur
new_index[0] = datetime.today().date()

# Réapplique l'index modifié
df.index = pd.to_datetime(new_index)

In [9]:
df

,tickers
2025-09-15,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA..."
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."
2007-02-14,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ..."


In [10]:

data_SP500_old = pd.read_parquet('/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_NASDAQ.parquet')
last_date = data_SP500_old['Date'].sort_values(ascending=True).iloc[-1] + timedelta(days=1)
last_date


Timestamp('2025-09-13 00:00:00')

In [11]:

df['Last_Date'] = last_date
df

,tickers,Last_Date
2025-09-15,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-13
2025-07-17,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-13
2025-05-19,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-13
2024-12-23,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-13
2024-11-18,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-13
...,...,...
2007-07-12,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-13
2007-06-01,"[A, AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMA...",2025-09-13
2007-03-08,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-13
2007-02-14,"[AAPL, ABNB, ADBE, ADI, ADP, ADSK, AEP, AMAT, ...",2025-09-13


In [12]:

# Initialiser une DataFrame vide pour stocker toutes les données
data_SP500 = pd.DataFrame()

df = df.explode('tickers')
end_date = df[df.index > last_date].groupby('tickers').apply(lambda x: x.index.max())

# Combiner les résultats dans une nouvelle DataFrame
resultat = pd.DataFrame({'Start': last_date, 'End': end_date})
resultat


/var/folders/6k/82j2nnl13hj9ld7nmzpdt6fh0000gn/T/ipykernel_3527/2328952231.py:5: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  end_date = df[df.index > last_date].groupby('tickers').apply(lambda x: x.index.max())


,Start,End
tickers,,
AAPL,2025-09-13,2025-09-15
ABNB,2025-09-13,2025-09-15
ADBE,2025-09-13,2025-09-15
ADI,2025-09-13,2025-09-15
ADP,2025-09-13,2025-09-15
...,...,...
VRTX,2025-09-13,2025-09-15
WBD,2025-09-13,2025-09-15
WDAY,2025-09-13,2025-09-15


In [13]:
"""
# Initialiser une DataFrame vide pour stocker toutes les données
data_SP500 = pd.DataFrame()

df = df.explode('tickers')
start_date = df.groupby('tickers').apply(lambda x: x.index.min())
end_date = df.groupby('tickers').apply(lambda x: x.index.max())

# Combiner les résultats dans une nouvelle DataFrame
resultat = pd.DataFrame({'Start': start_date, 'End': end_date})
resultat
"""

"\n# Initialiser une DataFrame vide pour stocker toutes les données\ndata_SP500 = pd.DataFrame()\n\ndf = df.explode('tickers')\nstart_date = df.groupby('tickers').apply(lambda x: x.index.min())\nend_date = df.groupby('tickers').apply(lambda x: x.index.max())\n\n# Combiner les résultats dans une nouvelle DataFrame\nresultat = pd.DataFrame({'Start': start_date, 'End': end_date})\nresultat\n"

In [14]:
# Télécharger les données pour chaque ticker
for ticker in resultat.index:
    start_date = resultat.loc[ticker, 'Start']
    end_date = resultat.loc[ticker, 'End']

    print(f"Téléchargement des données pour {ticker} de {start_date} à {end_date}")

    # Télécharger les données de l'action entre les dates spécifiées
    data = yf.download(
                        ticker,
                        start=start_date,
                        end=end_date,
                        interval="1d",
                        auto_adjust=True,
                        actions=False
                        )

    # Ajouter une colonne pour identifier le ticker
    data['Ticker'] = ticker

    # On retire le niveau des tickers pour ne garder que 'Close' et 'Volume'
    data.columns = data.columns.droplevel(level=1)
    
    # Concaténer les données dans la DataFrame globale
    data_SP500 = pd.concat([data_SP500, data])

# Affichage de la DataFrame résultante
data_SP500

Téléchargement des données pour AAPL de 2025-09-13 00:00:00 à 2025-09-15 00:00:00


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-09-13 00:00:00 -> 2025-09-15 00:00:00)')


Téléchargement des données pour ABNB de 2025-09-13 00:00:00 à 2025-09-15 00:00:00


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ABNB']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-09-13 00:00:00 -> 2025-09-15 00:00:00)')


Téléchargement des données pour ADBE de 2025-09-13 00:00:00 à 2025-09-15 00:00:00


...

Téléchargement des données pour XEL de 2025-09-13 00:00:00 à 2025-09-15 00:00:00


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['XEL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-09-13 00:00:00 -> 2025-09-15 00:00:00)')


Téléchargement des données pour ZS de 2025-09-13 00:00:00 à 2025-09-15 00:00:00


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ZS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2025-09-13 00:00:00 -> 2025-09-15 00:00:00)')


Price,Adj Close,Close,High,Low,Open,Volume,Ticker
Date,,,,,,,


In [15]:
data_SP500 = data_SP500.reset_index()
data_SP500

Price,Date,Adj Close,Close,High,Low,Open,Volume,Ticker


In [16]:
df_concat = pd.concat([data_SP500, data_SP500_old])
df_concat

Price,Date,Adj Close,Close,High,Low,Open,Volume,Ticker
0,2007-06-01,NaN,24.376429,24.427373,24.102609,24.306381,2656759.0,A
1,2007-06-04,NaN,24.510162,24.599314,24.191767,24.198134,2229950.0,A
2,2007-06-05,NaN,24.497437,24.554747,24.319134,24.331870,2744274.0,A
3,2007-06-06,NaN,24.433748,24.516532,24.293654,24.312758,2975224.0,A
4,2007-06-07,NaN,23.726902,24.433742,23.726902,24.274543,3567416.0,A
...,...,...,...,...,...,...,...,...
430769,2025-09-08,NaN,281.600006,281.760010,273.734009,275.000000,1839700.0,ZS
430770,2025-09-09,NaN,282.290009,284.339996,279.000000,281.350006,1634400.0,ZS
430771,2025-09-10,NaN,278.790009,288.890015,276.309998,284.299988,1836000.0,ZS
430772,2025-09-11,NaN,286.660004,292.940002,281.450012,282.049988,1946700.0,ZS


In [17]:
df_concat.sort_values(by=['Date'], ascending=True, inplace=True)
df_concat.tail(50)

Price,Date,Adj Close,Close,High,Low,Open,Volume,Ticker
114030,2025-09-12,NaN,48.400002,48.959999,48.380001,48.630001,3944900.0,CPRT
109346,2025-09-12,NaN,967.900024,969.570007,960.000000,960.289978,1460300.0,COST
53563,2025-09-12,NaN,582.000000,587.880005,572.109985,573.054993,6725900.0,APP
54064,2025-09-12,NaN,150.639999,155.550003,149.020004,155.240005,3601500.0,ARM
104662,2025-09-12,NaN,33.060001,33.380001,32.450001,33.270000,23848200.0,CMCSA
99978,2025-09-12,NaN,263.399994,266.130005,256.109985,264.000000,1764200.0,CHTR
96031,2025-09-12,NaN,323.480011,325.410004,315.281006,318.059998,1414800.0,CEG
95115,2025-09-12,NaN,164.289993,166.190002,163.929993,165.050003,1120700.0,CDW
427238,2025-09-12,NaN,73.349998,73.720001,72.599998,72.849998,7080600.0,XEL
94202,2025-09-12,NaN,343.480011,355.070007,342.089996,353.470001,1616900.0,CDNS


In [18]:
df_concat['Date'].max()

Timestamp('2025-09-12 00:00:00')

In [19]:
# Sauvegarder la DataFrame résultante dans un fichier CSV
#data_SP500.to_csv('/Users/forget/PyCharmMiscProject/historical_data_S&P500_From_Wiki.csv')

In [20]:
"""
mask_1 = data_SP500[(data_SP500['Ticker'] == 'NVR') & (data_SP500['Date'] < '1993-10-01')]
mask_2 = data_SP500[(data_SP500['Ticker'] == 'HUBB') & (data_SP500['Date'] < '1994-10-31')]
mask_3 = data_SP500[(data_SP500['Date'] == '1984-12-31') & (data_SP500['Ticker'] == 'BEN')]
"""

"\nmask_1 = data_SP500[(data_SP500['Ticker'] == 'NVR') & (data_SP500['Date'] < '1993-10-01')]\nmask_2 = data_SP500[(data_SP500['Ticker'] == 'HUBB') & (data_SP500['Date'] < '1994-10-31')]\nmask_3 = data_SP500[(data_SP500['Date'] == '1984-12-31') & (data_SP500['Ticker'] == 'BEN')]\n"

In [21]:
"""
idx_to_drop = mask_1.index.union(mask_2.index).union(mask_3.index)
data_SP500 = data_SP500.drop(index=idx_to_drop).copy()
"""

'\nidx_to_drop = mask_1.index.union(mask_2.index).union(mask_3.index)\ndata_SP500 = data_SP500.drop(index=idx_to_drop).copy()\n'

In [22]:
df_concat.to_parquet("/Users/forget/Library/Mobile Documents/com~apple~CloudDocs/Project Stock Market Deep Learning/Data/data_NASDAQ.parquet")